In [2]:
# imports
import os
import imp
import numpy as np
import pandas as pd
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bayes_mvs as bmvs

%matplotlib inline

sns.set_style('white')
sns.set_context('poster')
colors = sns.color_palette()

# aliases
L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor

In [3]:
# data loading
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
paramsdir = os.path.join(headdir, 'Analysis/0_hvh/Params/nnets/temp')
datadir = os.path.join(headdir, 'Data/model input')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nnets')
datafilenames = ['0 (with groups)', '1 (with computer)', '2 (with computer)', '3 (with computer)', '4']
datafilenames = [os.path.join(datadir, fname + '.csv') for fname in datafilenames]
colnames = ['subject', 'color', 'bp', 'wp', 'zet', 'rt']

e0 = pd.read_csv(datafilenames[0], names=colnames+['splitno'])
e1 = pd.read_csv(datafilenames[1], names=colnames)
e2 = pd.read_csv(datafilenames[2], names=colnames)
e3 = pd.read_csv(datafilenames[3], names=colnames+['task', 'taskorder', 'session'])
e4 = pd.read_csv(datafilenames[4], names=colnames+['timecondition'])
Es = [e1, e2, e3, e4]
for i, e in enumerate(Es[1:]):
    e['subject'] = e['subject'] + Es[i-1].loc[Es[i-1]['subject']<1000, 'subject'].max()

A = pd.concat([e[colnames] for e in [e1, e2, e3, e4]])

groups = np.arange(len(A))%5 + 1
np.random.seed(100001)
np.random.shuffle(groups)
A['group'] = groups

A.to_csv(os.path.join(datadir, '1-4.csv'), encoding='ASCII', header=False, index=False)
A.loc[A['subject']<1000, :].to_csv(
    os.path.join(datadir, '1-4 (no computer).csv'), 
    encoding='ASCII', header=False, index=False
)

In [ ]:
# class FineTuner(DefaultTrainer):
#     """
#     Trainer to fine tune networks to individual subjects

#     Consider moving freeze, param set functions properly into Network object
#     Abstracting split functions and augment in DefaultTrainer would be good too
#     """

#     def train_all(self, architecture, data, split, seed=None, startparams=None, freeze=True, save_params=False):
#         if seed:
#             np.random.seed(seed)

#         D, groups, Xs, ys, Ss = data
#         num_splits = len(Xs)
#         r = np.tile(np.arange(num_splits), [num_splits, 1])
#         r = (r + r.T) % num_splits

#         starttime = time.time()
#         net = Network(architecture)
#         if startparams:
#             _layers = L.get_all_layers(net.net)
#             L.set_all_param_values(_layers, startparams)
#             convlayer, prelulayer = _layers[1:3]
#             if freeze:
#                 convlayer.params[convlayer.W].remove('trainable')
#                 convlayer.params[convlayer.b].remove('trainable')
#                 prelulayer.params[prelulayer.alpha].remove('trainable')

#         train_idxs = r[split, :3]
#         val_idxs = r[split, 3:4]
#         test_idxs = r[split, 4:]

#         X, y, S = [np.concatenate(np.array(Z)[train_idxs]) for Z in [Xs, ys, Ss]]
#         Xv, yv, Sv = [np.concatenate(np.array(Z)[val_idxs]) for Z in [Xs, ys, Ss]]
#         Xt, yt, St = [np.concatenate(np.array(Z)[test_idxs]) for Z in [Xs, ys, Ss]]
#         X, y = augment((X, y))
#         S = np.concatenate([S, S, S, S])
#         self.train(net, training_data=(X, y), validation_data=(Xv, yv))
#         self.test(net, testing_data=(Xt, yt))
#         time_elapsed = time.time() - starttime

#         return net

In [4]:
data = loading.default_loader(os.path.join(datadir, '1-4 (no computer).csv'))
hvhdata = loading.default_loader(os.path.join(datadir, '0 (with groups).csv'))
Xs = np.concatenate(hvhdata[2])
ys = np.concatenate(hvhdata[3])
Ss = np.concatenate(hvhdata[4])

# Prototyping

REMEMBER: you need to replace `prototype` with `archX`

In [50]:
def run_full_fit(num_filters, pool_size=2, filter_size=(4,4)):
    """
    Runs the full fitting experiment, pretraining on later experiments and testing on first.
    Saves data as it goes to avoid eating memory.
    """
    # define architecture
    archname = 'arch{}'.format(num_filters)
    arch = lambda input_var: prototype(input_var, num_filters=num_filters, pool_size=pool_size, filter_size=filter_size)
    
    # start training
    trainer = DefaultTrainer(stopthresh=100)
    net_list = trainer.train_all(architecture=arch, data=data, seed=985227)
    
    # save params
    for i, n in enumerate(net_list):
        fname = '{} {} split agg fit exp 1-4'.format(archname, i)
        n.save_params(os.path.join(paramsdir, fname))
    
    # save high board-by-board nlls
        # WARNING: does not preserve board order!!!
    nlls, results = [], []
    for net in net_list:
        nll = net.test_fn(Xs, ys)
        nlls.append(nll[0])
        res = net.itemized_test_fn(Xs, ys)
        results.append(res)

    nlls_array = np.array(nlls)
    results_array = np.array(results)
    print("\nPretrain stats\n", bmvs(nlls_array))
    
    # save results per subject
    subresults = []
    for s in np.unique(Ss):
        idx = np.where(Ss==s)[0]
        res = results_array[:, idx].mean()
        subresults.append(res)
        
    subresults_array = np.array(subresults)
    fname = '{} agg fit exp 1-4 results by subject.csv'.format(archname)
    fname = os.path.join(resultsdir, fname)
    np.savetxt(fname, subresults_array, fmt='%.18f', delimiter=',')
        
    print("\nBy subject\n", bmvs(subresults_array, alpha=.95))
    
    # fine tune on hvh data
    
    tune_results = np.zeros([40, 5, 5])
    tuner = FineTuner(stopthresh=25)

    for i, n in enumerate(net_list):
        for j in range(5):
            fname = '{} {} agg fit exp 1-4 {} tune fit exp 0'.format(archname, i, j)
            params = L.get_all_param_values(n.net)
            subnet = tuner.train_all(architecture=arch, data=hvhdata, split=j, startparams=params, freeze=True)
            subnet.save_params(os.path.join(paramsdir, fname))
            res = subnet.itemized_test_fn(Xs, ys)
            
            for s in np.unique(Ss):
                idx = np.where(Ss==s)[0]
                tune_results[s, j, i] = res[idx].mean()
                
    fname = '{} agg fit exp 1-4 agg tune fit exp 0.csv'.format(archname)
    fname = os.path.join(resultsdir, fname)
    np.savetxt(fname, tune_results.reshape([40, 25]), fmt='%.18f', delimiter=',')
                
    print("\nFine tune result by subject\n", bmvs(tune_results.mean(axis=(1,2))))
    
    return None

In [ ]:
run_full_fit(4)


Split Number 0
(12362, 2, 4, 9)
Epoch 0 took 3.693s
	training loss:			2.8708
	validation loss:		2.7153
	validation accuracy:		16.73%
	total time elapsed:		3.789s
Epoch 50 took 3.407s
	training loss:			2.5360
	validation loss:		2.3982
	validation accuracy:		27.36%
	total time elapsed:		169.918s
Epoch 100 took 3.233s
	training loss:			2.5315
	validation loss:		2.3969
	validation accuracy:		27.49%
	total time elapsed:		335.583s
Epoch 150 took 2.964s
	training loss:			2.5317
	validation loss:		2.3876
	validation accuracy:		27.34%
	total time elapsed:		497.594s
Abandon ship!

TEST PERFORMANCE
	Stopped in epoch:		165
	Test loss:			2.3963
	Test accuracy:			27.61%

Split Number 1
(12396, 2, 4, 9)
Epoch 0 took 3.269s
	training loss:			2.8940
	validation loss:		2.7624
	validation accuracy:		16.16%
	total time elapsed:		3.362s
Epoch 50 took 3.040s
	training loss:			2.5418
	validation loss:		2.4054
	validation accuracy:		27.43%
	total time elapsed:		173.792s
Epoch 100 took 3.027s
	training loss:	

# Prototyping Tuning

### Pretraining

In [ ]:
trainer = DefaultTrainer(stopthresh=100) # default: 125
net_list = trainer.train_all(architecture=prototype, data=data, seed=985227)

for i, n in enumerate(net_list):
    n.save_params(os.path.join(paramsdir, '{} split agg fit exp 1-4'.format(i)))

In [ ]:
subresults = []
for s in np.unique(Ss):
    idx = np.where(Ss==s)[0]
    res = results_array[:, idx].mean()
    subresults.append(res)
bmvs(np.array(subresults), alpha=.95)

### Finetuning

In [ ]:
tune_results = np.zeros([40, 5, 5])
tuner = FineTuner(stopthresh=25)

for i, n in enumerate(net_list):
    for j in range(5):
        params = L.get_all_param_values(n.net)
        subnet = tuner.train_all(architecture=prototype, data=hvhdata, split=j, startparams=params, freeze=True)
        subnet.save_params(os.path.join(paramsdir, '{} agg fit exp 1-4 {} tune fit exp 0'.format(i, j)))
        
        res = subnet.itemized_test_fn(Xs, ys)
        for s in np.unique(Ss):
            idx = np.where(Ss==s)[0]
            tune_results[s, j, i] = res[idx].mean()